In [1]:
import sys
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sys.setrecursionlimit(10000)

In [3]:
def generate_sample_data():
    os.makedirs('content/training_data/class_1', exist_ok=True)
    os.makedirs('content/training_data/class_2', exist_ok=True)

    for i in range(10):
        img = Image.fromarray(np.random.randint(0,255,(224,224,3),dtype=np.uint8))
        img.save(f'content/training_data/class_1/img{i}.jpg')
        img.save(f'content/training_data/class_2/img{i}.jpg')

generate_sample_data()

In [4]:
base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

In [5]:
for layer in base_model.layers:
    layer.trainable = False

In [6]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256,activation='relu'),
    Dense(1, activation='sigmoid') # Changes for the number of classes you have
])

In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'content/training_data',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary')

Found 20 images belonging to 2 classes.


In [9]:
model.fit(train_generator,epochs=10)

Epoch 1/10


c:\Users\everton.bandeira\AppData\Local\pypoetry\Cache\virtualenvs\imb-ai-engineering-YFLZBaaa-py3.10\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.7046
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 807ms/step - accuracy: 0.5000 - loss: 8.0092
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step - accuracy: 0.5000 - loss: 3.5530
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 751ms/step - accuracy: 0.5000 - loss: 0.7009
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 794ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 872ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 850ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 766ms/step - accuracy: 0.5000 - loss: 0.6931
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 751ms/step - accuracy: 0.5000 - loss: 0.6931


# Fine-Tuning the pre_trained Model VGG16

In [10]:
# Unfreeze the top layers of the base model
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compile the model again
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model again
model.fit(train_generator,epochs=2)

Epoch 1/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.6931
Epoch 2/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.6931


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,142,661 (183.65 MB)

 Trainable params: 13,502,465 (51.51 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

 Optimizer params: 27,004,932 (103.02 MB)